## imports

In [1]:
from multiprocessing import Event, Lock, Pool, Queue, Value
import time

import cv2
import matplotlib.pyplot as plt
import numpy as np

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander
from cflib.positioning.motion_commander import MotionCommander

## constants

In [2]:
group_number = 4
uri = f'radio://0/{group_number}/2M'
camera = 2
delay_ms = 0
frame_buf_maxsize = 5
cmd_buf_maxsize = 10
buf_timeout = 1
log_period_ms = 50

## camera check

In [3]:
# cap = cv2.VideoCapture(camera)

# while(True):
#     # Capture frame-by-frame
#     ret, frame = cap.read()

#     # Compute
#     cv2.imshow('frame', frame)    

#     # Hit q to quit.
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the capture
# cap.release()
# cv2.destroyAllWindows()
# del cap

## cf helper functions

In [4]:
def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            print("{} {} {}".
                format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break

def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    
    wait_for_position_estimator(cf)
    time.sleep(0.1)

## init

In [5]:
def position_callback(timestamp, data, log_conf):
    with x_cur.get_lock():
        x_cur.value = data['kalman.stateX']
    with y_cur.get_lock():
        y_cur.value = data['kalman.stateY']
    with z_cur.get_lock():
        z_cur.value = data['kalman.stateZ']

def init(f_b, c_b, st, cf_, x_, y_, z_, cflock_):
    global frame_buf, cmd_buf, stop, cf, x_cur, y_cur, z_cur, cflock
    frame_buf = f_b
    cmd_buf = c_b
    stop = st
    cf = cf_
    x_cur = x_
    y_cur = y_
    z_cur = z_
    cflock = cflock_
#     start_time = time.perf_counter_ns()

## ascend

In [6]:
def ascend_and_hover():
    print('ascending')
            
    # Ascend:
    for y in range(10):
        with cflock:
            cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
        time.sleep(1)

    # Hover at 1 meter:
    for _ in range(20):
        with cflock:
            cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(1)

## capture

In [7]:
def capture_frames(camera, delay_ms):
    cap = cv2.VideoCapture(camera)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 2)
    
    while cap.isOpened() and not stop.is_set():
        ok, frame = cap.read()

        if ok:
            # TODO: make sure queue doesnt fill up
            frame_buf.put_nowait(frame)
        else:
            break

        time.sleep(delay_ms / 1000)
            
    cap.release()
    frame_buf.close()

## planning

In [8]:
def process_frame(buf_timeout):
    if failed:
        return
    
    while not stop.is_set():
        try:
            frame = frame_buf.get(timeout=buf_timeout)
        except Exception as e:
            break

        cmd_buf.put_nowait((0, 0, 1, 0))
        
    cmd_buf.close()

## command

In [9]:
def send_command(buf_timeout):
    while not stop.is_set():
        try:
            x, y, z, yaw = cmd_buf.get(timeout=buf_timeout)
        except Exception as e:
            break

        cf.commander.send_position_setpoint(x, y, z, yaw)

## descend

In [10]:
def hover_and_descend():
    print('descending')
    
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 10)
        time.sleep(0.1)
    
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)

## main loop

In [11]:
# TEMPORARY
def f():
    stop.set()

In [15]:
frame_buf = Queue(frame_buf_maxsize)
cmd_buf = Queue(cmd_buf_maxsize)
stop = Event()
x_cur = Value('f', 0)
y_cur = Value('f', 0)
z_cur = Value('f', 0)
cflock = Lock()

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('FAILED! FAILED! FAILED!')
    # TODO: just exit out

scf = SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache'))
scf.open_link()
cf = scf.cf

set_PID_controller(cf)

# log_conf = LogConfig(name='Position', period_in_ms=log_period_ms)
# log_conf.add_variable('kalman.stateX', 'float')
# log_conf.add_variable('kalman.stateY', 'float')
# log_conf.add_variable('kalman.stateZ', 'float')

# scf.cf.log.add_config(log_conf)
# log_conf.data_received_cb.add_callback(position_callback)
# log_conf.start()

# Ascend:
for y in range(10):
    cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
    time.sleep(0.1)

# Hover at 1 meter:
for _ in range(20):
    cf.commander.send_hover_setpoint(0, 0, 0, 1)
    time.sleep(0.1)
    
for y in range(10):
    cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 10)
    time.sleep(0.1)

for i in range(10):
    cf.commander.send_stop_setpoint()
    time.sleep(0.1)

# pool = Pool(1, initializer=init, initargs=(frame_buf, cmd_buf, stop, cf, x_cur, y_cur, z_cur, cflock))

# pool.apply(ascend_and_hover)

# capture_frames_future = pool.apply_async(capture_frames, args=(camera, delay_ms))
# process_frame_future = pool.apply_async(process_frame, args=(buf_timeout,))
# send_command_future = pool.apply_async(send_command, args=(buf_timeout,))

# time.sleep(2)
# pool.apply(f)

# capture_frames_future.get()
# process_frame_future.get()
# send_command_future.get()

# pool.apply(hover_and_descend)

# pool.close()
# pool.join()
scf.close_link()

Scanning interfaces for Crazyflies...


Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'


Crazyflies found:
radio://0/4/2M
radio://0/4/2M
radio://0/4/2M
radio://0/4/2M
Initializing PID Controller
Waiting for estimator to find position...
999.9999879814131 999.9999879813467 999.9996973859961
999.9999909447242 999.9999909448898 999.9998196867964
999.9999909447242 999.9999909448898 999.9998196867964
999.9999910405122 999.9999910405568 999.9998196867964
999.9999910405122 999.9999910405568 999.9998207527242
999.9999910405122 999.9999910405568 999.9998207527242
999.9999910405122 999.9999910405568 999.9998207527242
999.9999910405122 999.9999910405568 999.9998207527242
999.9999910405122 999.9999910405568 999.9998207527242
3.0590990718337707e-06 3.059210030187387e-06 0.00012962422624696046


# TODO

* check cf init
* actually think about detection
* keep track of position?